<a href="https://colab.research.google.com/github/MayyarAA/NaturalLangaugeProcessingBERT/blob/main/MSCI598ProjectBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [ ]:
!pip install transformers

In [ ]:
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import matplotlib.pyplot as plt
import sklearn    
from sklearn.model_selection import train_test_split

import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
classifier("Great! I'm so happy this exists.")

[{'label': 'POSITIVE', 'score': 0.9998798370361328}]

In [ ]:
classifier("I was so bad I can't tell!")

[{'label': 'NEGATIVE', 'score': 0.9997102618217468}]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')     


Mounted at /content/drive


In [ ]:
train_body = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")

In [ ]:
train_body = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")
train_stance = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_stances.csv")

In [ ]:
train_body.head(5)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [ ]:
train_stance.head(5)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [ ]:
train_stance.loc[train_stance['Body ID']==712].head(2)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1787,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss


In [ ]:
full_sb = pd.merge(train_stance,train_body,on="Body ID")
full_sb.head(1)

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [ ]:
import re
def removeSpecailCharFromString(orgWord):
    if(len(orgWord)>100):
      orgWord = orgWord[:100];
    orgWord = orgWord.rstrip('\n')
    return re.sub(r"[^a-zA-Z0-9]+", ' ', orgWord).strip()

In [ ]:
inputs = tokenizer(full_sb.iloc[0].Headline, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2610,  2424,  3742,  9729,  2007,  2012,  2560,  1005,  2321,
          4230,  1005,  2379,  3290,  2237,  2073,  4724,  2493,  5419,  2044,
          2610, 13249,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
### using simple BERT
!pip install simpletransformers
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)

In [ ]:
# Preparing train data
train_data = [
    [
        "Aragorn was the heir of Isildur",
        "Gimli fought with a battle axe",
        1,
    ],
    [
        "Frodo was the heir of Isildur",
        "Legolas was an expert archer",
        0,
    ],
]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text_a", "text_b", "labels"]

# Preparing eval data
eval_data = [
    [
        "Theoden was the king of Rohan",
        "Gimli's preferred weapon was a battle axe",
        1,
    ],
    [
        "Merry was the king of Rohan",
        "Legolas was taller than Gimli",
        0,
    ],
]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text_a", "text_b", "labels"]

# Optional model configuration
#model_args = ClassificationArgs(num_train_epochs=1)


In [ ]:
from scipy.stats import pearsonr, spearmanr

def pearson_corr(preds, labels): return pearsonr(preds, labels)[0]

def spearman_corr(preds, labels): return spearmanr(preds, labels)[0]

In [ ]:
print(len(full_sb.iloc[0]['articleBody']))

1093


In [ ]:
#make full_sb shorter body

full_sb['articleBody'] = full_sb['articleBody'].str.slice(0,50)
full_sb.head(1)

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [ ]:
print(len(full_sb.iloc[0]['articleBody']))

50


In [ ]:
train_df_simple = full_sb[['Headline', 'articleBody','Stance']].copy()
train_df_simple.columns = ["text_a", "text_b", "labels"]
train_df_simple.head(1)
smaller_sb = full_sb.iloc[0:1000]
print(len(smaller_sb))

1000


In [ ]:
train_df_simple = smaller_sb[['Headline', 'articleBody','Stance']].copy()

In [ ]:
print(train_df_simple.iloc[0])

Headline       Police find mass graves with at least '15 bodi...
articleBody    Danny Boyle is directing the untitled film\n\n...
Stance                                                 unrelated
Name: 0, dtype: object


In [ ]:
# Create a ClassificationModel
model_args = {
    "num_train_epochs": 5,
    #"learning_rate": 1e-4,
    "overwrite_output_dir": True
}

model = ClassificationModel("roberta", "roberta-base",args=model_args)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
#model.train_model(train_df=train_df_simple, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)
model.train_model(train_df_simple)

RuntimeError: ignored

In [ ]:
model.train_model(train_df, eval_df=eval_df, pearson_corr=pearson_corr, spearman_corr=spearman_corr)

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


RuntimeError: ignored

In [ ]:
# Create a ClassificationModel
model_args = {
    "num_train_epochs": 5,
    "learning_rate": 1e-4,
    "overwrite_output_dir": True
}

model = ClassificationModel("roberta", "roberta-base",args=model_args)

# Train the model
#model.train_model(train_df).format(overwrite_output_dir=True)
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    eval_df
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

RuntimeError: ignored

In [ ]:
print(f'result {result}')
print(f'model_outputs {model_outputs}')

result {'mcc': 0.0, 'tp': 0, 'tn': 1, 'fp': 0, 'fn': 1, 'auroc': 1.0, 'auprc': 1.0, 'eval_loss': 0.656494140625}
model_outputs [[-0.04705811 -0.07958984]
 [ 0.02758789 -0.16040039]]


In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(
    [
        [
            "Legolas was an expert archer",
            "Legolas was taller than Gimli",
        ],[
            "Legolas was an expert archer",
            "Legolas was an expert archer",
        ],["hello","hello"],
     ["Theoden was the king of Rohan",
        "Gimli's preferred weapon was a battle axe"]
    ]
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(f'predictions {predictions}')

predictions [0 0 0 1]


In [ ]:
#data cleaning


#tokenize use BERT

#load model
#fine tune model 